In [45]:
import requests
import re
from bs4 import BeautifulSoup

In [ ]:
variant_template = {
	'size': 'Medium',
	'units': 0,
	'waist_low': 10,
	'waist_high': 11,
}

updated_items = {
	'price': 0.00,
	'in_stock': False,
	'ml_per_unit_price': 0.00,
	'total_price': 0.00,
	'unit_price': 0.00,
}

In [ ]:
urls = [
	'https://us.abuniverse.com/product/agz/'
]

In [83]:
stored_products = {
    'https://myinnerbaby.com/products/ageplay-outfitters-seaside-princess-printed-adult-diaper.json': {
        'template': {
            'backing': 'plastic',
            'brand': 'MyInnerBaby',
            'capacity': 5000,
            'name': 'Seaside Princess',
            'notes': '',
            'retailer': 'MyInnerBaby',
            'shipping': 0.00,
            'tapes': '4',
            'url': 'https://bambinodiapers.com/products/new-cloudee-all-over-print-diapers'
        },
        'sizes': {
            'Large (L)': {
                'label': 'Large',
                'waist_high': 50,
                'waist_low': 36,
            },
            'Medium (M)': {
                'label': 'Medium',
                'waist_high': 40,
                'waist_low': 32,
            },
        }

    },
    'https://bambinodiapers.com/products/new-cloudee-all-over-print-diapers.json': {
        'template': {
            'backing': 'plastic',
            'brand': 'Bambino',
            'capacity': 5000,
            'name': 'Cloudee',
            'notes': '',
            'retailer': 'Bambino',
            'shipping': 0.00,
            'tapes': '4',
            'url': 'https://bambinodiapers.com/products/new-cloudee-all-over-print-diapers'
        },
        'sizes': {
            'L': {
                'label': 'Large',
                'waist_high': 50,
                'waist_low': 38
            },
            'M': {
                'label': 'Medium',
                'waist_high': 40,
                'waist_low': 32
            },
            'XL': {
                'label': 'XLarge',
                'waist_high': 58,
                'waist_low': 40
            },
        }
    },
    'https://tykables.com/products/str8up-pink.json': {
        'template': {
            'backing': 'plastic',
            'brand': 'Tykables',
            'capacity': 8000,
            'name': 'Str8up Pink',
            'notes': '',
            'retailer': 'Tykables',
            'shipping': 0.00,
            'tapes': '4',
            'url': 'https://tykables.com/products/str8up-pink'
        },
        'sizes': {
            'Large': {
                'label': 'Large',
                'waist_high': 48,
                'waist_low': 34,
            },
            'Medium': {
                'label': 'Medium',
                'waist_high': 36,
                'waist_low': 28,
            },
            'XL': {
                'label': 'XLarge',
                'waist_high': 52,
                'waist_low': 48,
            }
        }
    },
    'https://us.abuniverse.com/product/agz/': {
        'id': '356230',
		'template': {
            'backing': 'cloth',
            'brand': 'ABU',
            'capacity': 7500,
            'name': 'ABU AlphaGatorZ',
            'notes': '',
            'retailer': 'ABU',
            'shipping': 0.00,
            'tapes': '4',
            'url': 'https://us.abuniverse.com/product/agz/'
        },
        'sizes': {
            'l': {
                'label': 'Large',
                'waist_high': 42,
                'waist_low': 37
            },
            'm': {
                'label': 'Medium',
                'waist_high': 36,
                'waist_low': 31
            },
            'xl': {
                'label': 'XLarge',
                'waist_high': 48,
                'waist_low': 43
            },
        },
        'units': {
            10: 'abu_quantity_1',
            40: 'abu_quantity_2',
            80: 'abu_quantity_3',
        },
	},
    'https://www.littleforbig.com/wp-json/wc/store/products/155255': {
        'id': '155255',
		'template': {
            'backing': 'plastic',
            'brand': 'LittleForBig',
            'capacity': 5352,
            'name': 'Blushing Baby',
            'notes': 'assumed ISO',
            'retailer': 'LittleForBig',
            'shipping': 0.00,
            'tapes': '4',
            'url': 'https://www.littleforbig.com/product/blushing-baby-adult-diapers-10-pieces-packm-l/'
        },
        'sizes': {
            'Size L': {
                'label': 'Large',
                'waist_high': 46,
                'waist_low': 36
            },
            'Size M': {
                'label': 'Medium',
                'waist_high': 38,
                'waist_low': 28
            },
        },
	},
}

In [24]:
def get_response(url):
	with requests.get(url) as response:
		if response.status_code != 200:
			print(f'Error: {response.status_code} for {url}')
			return
		return response

def get_soup(url):
		return BeautifulSoup(get_page(url), 'html.parser')

def get_page(url):
	return get_response(url).text

def get_data(url):
	return get_response(url).json()

In [61]:
# ABU
url = 'https://us.abuniverse.com/product/agz/'
rows = []
soup = get_soup(url)
stored_product = stored_products[url]
base_items = stored_product['template']

id = stored_product['id']

for variation in get_data(f'https://us.abuniverse.com/wp-json/wc/store/products/{id}')['variations']:
	attributes = variation['attributes']

	if attributes[2]['value'] == 'scent' or attributes[0]['value'] == 'sample':
		continue

	variation_id = variation['id']
	data = get_data(f'https://us.abuniverse.com/wp-json/wc/store/products/{variation_id}')

	stored_size = stored_product['sizes'][attributes[1]['value']]

	size_label = stored_size['label']
	waist_low = stored_size['waist_low']
	waist_high = stored_size['waist_high']

	for units, ident in stored_product['units'].items():
		price_data = soup.find('label', {'for': ident}).string
		price = float(re.search('\d*\.\d*', price_data).group())
		unit_price = float(price / units)
		ml_per_unit_price = int(base_items['capacity'] / unit_price)
		total_price = price + base_items['shipping']
		in_stock = 'Yes' if data['is_in_stock'] else 'No'
		additional_items = {
			'size_label': size_label,
			'units': units,
			'waist_low': waist_low,
			'waist_high': waist_high,
			'price': price,
			'in_stock': in_stock,
			'ml_per_unit_price': ml_per_unit_price,
			'total_price': total_price,
			'unit_price': unit_price,
		}
		rows.append(base_items | additional_items)


print(*rows, sep='\n')

{'backing': 'cloth', 'brand': 'ABU', 'capacity': 7500, 'name': 'ABU AlphaGatorZ', 'notes': '', 'retailer': 'ABU', 'shipping': 0.0, 'tapes': '4', 'url': 'https://us.abuniverse.com/product/agz/', 'label': 'Medium', 'units': 10, 'waist_low': 31, 'waist_high': 36, 'price': 43.99, 'in_stock': True, 'ml_per_unit_price': 1704, 'total_price': 43.99, 'unit_price': 4.399}
{'backing': 'cloth', 'brand': 'ABU', 'capacity': 7500, 'name': 'ABU AlphaGatorZ', 'notes': '', 'retailer': 'ABU', 'shipping': 0.0, 'tapes': '4', 'url': 'https://us.abuniverse.com/product/agz/', 'label': 'Medium', 'units': 40, 'waist_low': 31, 'waist_high': 36, 'price': 143.99, 'in_stock': True, 'ml_per_unit_price': 2083, 'total_price': 143.99, 'unit_price': 3.5997500000000002}
{'backing': 'cloth', 'brand': 'ABU', 'capacity': 7500, 'name': 'ABU AlphaGatorZ', 'notes': '', 'retailer': 'ABU', 'shipping': 0.0, 'tapes': '4', 'url': 'https://us.abuniverse.com/product/agz/', 'label': 'Medium', 'units': 80, 'waist_low': 31, 'waist_high'

In [73]:
#Bambino
url = 'https://bambinodiapers.com/products/new-cloudee-all-over-print-diapers.json'
rows = []
product = get_data(url)['product']
stored_product = stored_products[url]
base_items = stored_product['template']

for variant in product['variants']:
	if variant['option2'].startswith('1 Sample'):
		continue

	size = variant['option1'][:-10]
	stored_size = stored_product['sizes'][size]
	size_label = stored_size['label']
	waist_high = stored_size['waist_high']
	waist_low = stored_size['waist_low']
	units = int(variant['option2'][-2:])
	price = float(variant['price'])
	in_stock = 'Maybe'
	unit_price = float(price / units)
	ml_per_unit_price = int(base_items['capacity'] / unit_price)
	total_price = float(price + base_items['shipping'])

	additional_items = {
		'size_label': size_label,
		'units': units,
		'waist_low': waist_low,
		'waist_high': waist_high,
		'price': price,
		'in_stock': in_stock,
		'ml_per_unit_price': ml_per_unit_price,
		'total_price': total_price,
		'unit_price': unit_price,
	}

	rows.append(base_items | additional_items)

print(*rows, sep='\n')

{'backing': 'plastic', 'brand': 'Bambino', 'capacity': 5000, 'name': 'Cloudee', 'notes': '', 'retailer': 'Bambino', 'shipping': 0.0, 'tapes': '4', 'url': 'https://bambinodiapers.com/products/new-cloudee-all-over-print-diapers', 'size_label': 'Medium', 'units': 8, 'waist_low': 32, 'waist_high': 40, 'price': 30.99, 'in_stock': 'Maybe', 'ml_per_unit_price': 1290, 'total_price': 30.99, 'unit_price': 3.87375}
{'backing': 'plastic', 'brand': 'Bambino', 'capacity': 5000, 'name': 'Cloudee', 'notes': '', 'retailer': 'Bambino', 'shipping': 0.0, 'tapes': '4', 'url': 'https://bambinodiapers.com/products/new-cloudee-all-over-print-diapers', 'size_label': 'Medium', 'units': 48, 'waist_low': 32, 'waist_high': 40, 'price': 119.99, 'in_stock': 'Maybe', 'ml_per_unit_price': 2000, 'total_price': 119.99, 'unit_price': 2.4997916666666664}
{'backing': 'plastic', 'brand': 'Bambino', 'capacity': 5000, 'name': 'Cloudee', 'notes': '', 'retailer': 'Bambino', 'shipping': 0.0, 'tapes': '4', 'url': 'https://bambino

In [93]:
#LittleForBig
url = 'https://www.littleforbig.com/wp-json/wc/store/products/155255'
rows = []
product = get_data(url)
stored_product = stored_products[url]
base_items = stored_product['template']

for variation in product['variations']:
	variant_id = variation['id']
	print(variant_id)
	variant = variation['attributes'][0]['value']
	variant_data = get_data(f'https://www.littleforbig.com/wp-json/wc/store/products/{variant_id}')

	size = variant[:6]
	stored_size = stored_product['sizes'][size]
	size_label = stored_size['label']
	waist_low = int(stored_size['waist_low'])
	waist_high = int(stored_size['waist_high'])
	units = variant[10:12]
	print(units)
	price = float(variant['price'])
	in_stock = 'Maybe'
	unit_price = float(price / units)
	ml_per_unit_price = int(base_items['capacity'] / unit_price)
	total_price = float(price + base_items['shipping'])

	additional_items = {
		'size_label': size_label,
		'units': units,
		'waist_low': waist_low,
		'waist_high': waist_high,
		'price': price,
		'in_stock': in_stock,
		'ml_per_unit_price': ml_per_unit_price,
		'total_price': total_price,
		'unit_price': unit_price,
	}
	
	rows.append(base_items | additional_items)

print(*rows, sep='\n')

155309


ValueError: invalid literal for int() with base 10: 'f '

In [81]:
#MyInnerbaby
url = 'https://myinnerbaby.com/products/ageplay-outfitters-seaside-princess-printed-adult-diaper.json'
rows = []
product = get_data(url)['product']
stored_product = stored_products[url]
base_items = stored_product['template']

for variant in product['variants']:
	if variant['option2'] != 'Bag of 10' and variant['option2'] != 'Case of 40':
		continue

	size = variant['option1']
	stored_size = stored_product['sizes'][size]
	size_label = stored_size['label']
	waist_low = int(stored_size['waist_low'])
	waist_high = int(stored_size['waist_high'])
	units = int(variant['option2'][-2:])
	price = float(variant['price'])
	in_stock = 'Maybe'
	unit_price = float(price / units)
	ml_per_unit_price = int(base_items['capacity'] / unit_price)
	total_price = float(price + base_items['shipping'])

	additional_items = {
		'size_label': size_label,
		'units': units,
		'waist_low': waist_low,
		'waist_high': waist_high,
		'price': price,
		'in_stock': in_stock,
		'ml_per_unit_price': ml_per_unit_price,
		'total_price': total_price,
		'unit_price': unit_price,
	}
	
	rows.append(base_items | additional_items)

print(*rows, sep='\n')

{'backing': 'plastic', 'brand': 'MyInnerBaby', 'capacity': 5000, 'name': 'Seaside Princess', 'notes': '', 'retailer': 'MyInnerBaby', 'shipping': 0.0, 'tapes': '4', 'url': 'https://bambinodiapers.com/products/new-cloudee-all-over-print-diapers', 'size_label': 'Medium', 'units': 10, 'waist_low': 32, 'waist_high': 40, 'price': 39.95, 'in_stock': 'Maybe', 'ml_per_unit_price': 1251, 'total_price': 39.95, 'unit_price': 3.995}
{'backing': 'plastic', 'brand': 'MyInnerBaby', 'capacity': 5000, 'name': 'Seaside Princess', 'notes': '', 'retailer': 'MyInnerBaby', 'shipping': 0.0, 'tapes': '4', 'url': 'https://bambinodiapers.com/products/new-cloudee-all-over-print-diapers', 'size_label': 'Medium', 'units': 40, 'waist_low': 32, 'waist_high': 40, 'price': 109.95, 'in_stock': 'Maybe', 'ml_per_unit_price': 1819, 'total_price': 109.95, 'unit_price': 2.7487500000000002}
{'backing': 'plastic', 'brand': 'MyInnerBaby', 'capacity': 5000, 'name': 'Seaside Princess', 'notes': '', 'retailer': 'MyInnerBaby', 'shi

In [62]:
# Tykables
url = 'https://tykables.com/products/str8up-pink.json'
rows = []
product = get_data(url)['product']
stored_product = stored_products[url]
base_items = stored_product['template']

for variant in product['variants']:
	if variant['option3'] != 'None' or variant['option2'] == '1 Sample':
		continue

	size = variant['option1']
	stored_size = stored_product['sizes'][size]
	size_label = stored_size['label']
	waist_low = int(stored_size['waist_low'])
	waist_high = int(stored_size['waist_high'])
	units = int(variant['option2'][:2])
	price = float(variant['price'])
	in_stock = 'Yes' if variant['inventory_quantity'] > 0 else 'No'
	unit_price = float(price / units)
	ml_per_unit_price = int(base_items['capacity'] / unit_price)
	total_price = float(price + base_items['shipping'])

	additional_items = {
		'size_label': size_label,
		'units': units,
		'waist_low': waist_low,
		'waist_high': waist_high,
		'price': price,
		'in_stock': in_stock,
		'ml_per_unit_price': ml_per_unit_price,
		'total_price': total_price,
		'unit_price': unit_price,
	}
	
	rows.append(base_items | additional_items)

print(*rows, sep='\n')

{'backing': 'plastic', 'brand': 'Tykables', 'capacity': 8000, 'name': 'Str8up Pink', 'notes': '', 'retailer': 'Tykables', 'shipping': 0.0, 'tapes': '4', 'url': 'https://tykables.com/products/str8up-pink', 'size_label': 'Medium', 'units': 10, 'waist_low': 28, 'waist_high': 36, 'price': 47.0, 'in_stock': True, 'ml_per_unit_price': 1702, 'total_price': 47.0, 'unit_price': 4.7}
{'backing': 'plastic', 'brand': 'Tykables', 'capacity': 8000, 'name': 'Str8up Pink', 'notes': '', 'retailer': 'Tykables', 'shipping': 0.0, 'tapes': '4', 'url': 'https://tykables.com/products/str8up-pink', 'size_label': 'Medium', 'units': 40, 'waist_low': 28, 'waist_high': 36, 'price': 125.0, 'in_stock': True, 'ml_per_unit_price': 2560, 'total_price': 125.0, 'unit_price': 3.125}
{'backing': 'plastic', 'brand': 'Tykables', 'capacity': 8000, 'name': 'Str8up Pink', 'notes': '', 'retailer': 'Tykables', 'shipping': 0.0, 'tapes': '4', 'url': 'https://tykables.com/products/str8up-pink', 'size_label': 'Medium', 'units': 80, 